In [1]:
import pandas as pd

In [3]:
from GNOD_Functions import billboard_scraper
from GNOD_Functions import basic_recommendation_engine

In [ ]:
client_id= getpass.getpass()
client_secret=getpass.getpass()

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

## 1. Basic recommendation engine

In [154]:
def GNOD_recommender():
    
    billboard = billboard_scraper()
    
    song = input("What is the name of your song?").lower().replace(" ", '')

    #get the billboard record - if available
    check = billboard[billboard['song'].str.lower().str.replace(" ","").str.contains(song)]

    #get the index of the song in the entry
    index = check.index.tolist()

    # check if the check has returned a value or not (is the song hot or not?)
    if len(check) == 0:
        cluster = clustering()
        
        spotify_recommendation(cluster)
        
    else:
        answer = input("Do you mean " + billboard.song[index].values[0] + " by " + billboard.artist[index].values[0] + "?")
    # provide a suggestion in case the song is the list    
        if answer.lower() == 'yes':
            suggestion = billboard.sample().index.tolist()
            print("Nice! This is a hot song! You might also like " + billboard['song'][suggestion].item() + " by " + billboard['artist'][suggestion].item())
        else:
            print("Well, not so hot after all. We were thinking about different tracks.")

In [155]:
GNOD_recommender()

  0%|          | 0/100 [00:00<?, ?it/s]

What is the name of your song?Loose Yourself
 How about trying out Sing About Me, I'm Dying Of Thirst by Kendrick Lamar?


In [143]:
def clustering():

    import warnings
    warnings.filterwarnings("ignore")
    from sklearn.cluster import KMeans
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    import getpass
    from sklearn.preprocessing import StandardScaler

    tracks = pd.read_csv('tracks.csv')

    # load and scale the dataframe with all track details
    numeric = tracks.select_dtypes(exclude = 'object')
    scaler = StandardScaler().fit(numeric)
    scaled = scaler.fit_transform(numeric)
    numeric_scaled = pd.DataFrame(scaled)

    # construct the K means prediction model and reference df
    kmeans = KMeans(n_clusters = 39, random_state=40).fit(numeric_scaled)
    clusters = kmeans.predict(numeric_scaled)
    track_cluster = pd.DataFrame(tracks)
    track_cluster['cluster'] = clusters
    
    return kmeans, track_cluster

In [61]:
numeric.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.000,0.385,3,-18.509,0,0.0000,0.23800,0.97800,0.0000,0.000,0.000,6453
1,0.680,0.767,11,-2.387,0,0.0397,0.00213,0.00000,0.0996,0.344,82.518,179467
2,0.538,0.675,0,-2.300,1,0.3350,0.24500,0.00000,0.3620,0.519,78.985,288880
3,0.653,0.826,6,-2.930,0,0.3540,0.01530,0.00146,0.1310,0.416,108.520,256227
4,0.902,0.720,6,-2.776,0,0.3470,0.26000,0.00000,0.0749,0.805,90.059,193467


In [149]:
suggestion = spotify_recommendation(clustering())

 How about trying out DontGetIt by Lil Wayne?


In [47]:
features = pd.DataFrame(sp.audio_features(song_id)).drop(columns = ['type','id','uri','track_href','analysis_url', 'time_signature'])

In [52]:
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.759,0.459,8,-5.187,1,0.0948,0.00323,0,0.0906,0.695,109.997,164442


In [148]:
def spotify_recommendation(clustering):

    # compare user input

    #song_id = sp.search(q = song, type = 'track', limit=1)['tracks']['items'][0]['uri']
    #song_check = sp.search(q = song, type = 'track', limit=1)['tracks']['items'][0]['name']
    artist_check = sp.search(q = song, type = 'track', limit=1)['tracks']['items'][0]['artists'][0]['name']

    #answer = input("Do you mean " + song_check  + " by " + artist_check + "?")

    #if answer == 'yes':
    features = pd.DataFrame(sp.audio_features(song_id)).drop(columns = ['type','id','uri','track_href','analysis_url', 'time_signature'])
    prediction = kmeans.predict(features)[0]
    suggestion = track_cluster[track_cluster.cluster == prediction].sample(1)
    suggestion = suggestion['name'].values[0]
    artist = sp.search(q = suggestion, type = 'track', limit=1)['tracks']['items'][0]['artists'][0]['name']
    track_cluster.head()
    print(" How about trying out " + str(suggestion) + " by " + str(artist) + "?")

In [138]:
suggestion = track_cluster[track_cluster.cluster == prediction].sample(1)
suggestion = suggestion['name'].values[0]
suggestion

'Bop Gun (One Nation) - Remastered'

In [139]:
sp.search(q = suggestion, type = 'track', limit=1)['tracks']['items'][0]['artists'][0]['name']

'Ice Cube'